In [1]:
#object that holds a lot of our data

class NewsObject:
    id = '0'
    targetTitle = []
    targetDescription = ""
    targetKeywords = []
    targetParagraphs = []
    targetCaptions = []
    postText = []
    postMedia = []
    postTimestamp = ''
    #truthMedian = ''
    #truthMean = ''
    #truthMode = ''
    truthClass = ""
    #truthJudgments = []
    attributes = ()
    
    def __init__(self, line):
        
        self.id = line['id']
        self.targetTitle= line['targetTitle']
        self.targetKeywords = line['targetKeywords']
        self.targetParagraphs = line['targetParagraphs']
        self.targetCaptions = line['targetCaptions']
        self.postText = line['postText']
        self.postMedia = line['postMedia']
        self.postTimestamp = line['postTimestamp']
        
    def addTruth(self, line):
        #self.truthMedian = line['truthMedian']
        #self.truthMean = line['truthMean']
        #self.truthMode = line['truthMode']
        if line['truthClass'] == 'clickbait':
            self.truthClass = '1'
        else:
            self.truthClass = '0'
        #self.truthJudgments = line['truthJudgments']
        


In [2]:
import nltk 
from nltk.corpus import stopwords
import json
import arff # https://pypi.python.org/pypi/liac-arff
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

contractionsFile = open("english-contractions-list.txt", "r")
contractions = []
for line in contractionsFile:
    contractions = line.split(',')

stopWords = set(stopwords.words('english'))


C:\Users\Philz zee Kill\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
#import files
instances = []

with open('dataset/instances_train.jsonl') as file:
    for line in file:
        temp = NewsObject(json.loads(line))
        instances.append(temp)
        
with open('dataset/truth_train.jsonl') as file2:
    i = 0
    for line in file2:
        instances[i].addTruth(json.loads(line))
        i += 1
#print(instances[0].targetKeywords)

In [35]:
def extractFeatures(newsObject):
    feat = {
        "wordCount": 0, "informal": 0, "beginsQuestion": False, "beginsNum": False,
        "beginsThis": False, "titleStopPerc": 0, "titleProperPerc": 0, "posSent": 0,
        "neuSent": 0, "negSent": 0, "compoundSent": 0, "articleWords": 0, "titlePercVerbs": 0,
        "unigrams": 0, "bigrams": 0, "trigrams": 0, "fourgrams": 0, "fivegrams": 0, "percNouns": 0,"unigramsArticle": 0,
        "bigramsArticle": 0, "trigramsArticle": 0, "fourgramsArticle": 0, "fivegramsArticle": 0, "percAdj": 0,
        "percAdv": 0,  "percentKeywordsInTitle": 0, "sentDiffTitleBody": 0, "hasContractions": False, "hasAllCaps": False,
        "averageWordLength": 0, "percSimilarToTitle": 0, "hasThis": False, "hasQuestion": False, "hasNumber": False,
        "label": newsObject.truthClass
    }
    
    words = newsObject.postText[0].split(' ')
    keywords = newsObject.targetKeywords
    text = nltk.word_tokenize(newsObject.postText[0])
    tokenizedList = nltk.pos_tag(text)    
    
    feat['wordCount'] = len(words)
    
    if words[0].isdigit():
        feat['beginsNum'] = True
        feat['hasNumber'] = True
    elif words[0].lower() == 'this':
        feat['beginsThis'] = True
        feat['hasThis'] = True
    
    numSame = 0
    index = 0
    length = 0
    for word in words:
        if word in keywords:
            numSame += 1
        if word in contractions:
            feat['hasContractions'] = True
        if len(word) > 5 and "#" not in word and "@" not in word and index > 0 and word.isupper():
            feat['hasAllCaps'] = True
        index += 1
        length += len(word)
        feat['averageWordLength'] = round(length/len(words), 2)
                    
                    
    originalTitle = newsObject.targetTitle.split()
    numSimilar = 0
    for word in words:
        if word in originalTitle:
            numSimilar += 1
    
    feat['percSimilarToTitle'] = round(numSimilar/feat['wordCount'], 2)
            
  
    numProper = 0
    numStop = 0
    numVerb = 0
    numNoun = 0
    numAdj = 0
    numAdv = 0
    
    if len(tokenizedList) > 0:
        firstWord = tokenizedList[0]
        if firstWord[1] == 'MD' or firstWord[1] == 'WRB':
            feat['beginsQuestion'] = True
            feat['hasQuestion'] = True
    
    for partOfSpeech in tokenizedList:
        if feat['hasThis'] == False and partOfSpeech[0] == 'this':
            feat['hasThis'] = True
        if partOfSpeech[1] == 'NNP':
            numProper += 1
        elif partOfSpeech[1] == 'VB' or partOfSpeech[1] == 'VBP' or partOfSpeech[1] == 'VBD' or partOfSpeech[1] == 'VBN':
            numVerb += 1
        elif partOfSpeech[1] == 'NN':
            numNoun += 1
        elif partOfSpeech[1] == 'PRP':
            feat['informal'] += 1
        elif partOfSpeech[1] == 'JJ':
            numAdj += 1
        elif partOfSpeech[1] == 'RB':
            numAdv += 1
        elif feat['hasNumber'] == False and partOfSpeech[1] == 'CD':
            feat['hasNumber'] = True
        elif feat['hasQuestion'] == False and partOfSpeech[1] == 'MD' or partOfSpeech[1] == 'WRB':
            feat['hasQuestion'] = True
        if(partOfSpeech[0] in stopWords):
            numStop += 1

    feat['titleStopPerc'] = round(numStop/feat['wordCount'], 2)
    feat['titleProperPerc'] = round(numProper/feat['wordCount'], 2)
    feat['titlePercVerbs'] = round(numVerb/feat['wordCount'], 2)
    feat['percNouns'] = round(numNoun/feat['wordCount'], 2)
    feat['percAdj'] = round(numAdj/feat['wordCount'], 2)
    feat['percAdv'] = round(numAdv/feat['wordCount'], 2)
    feat['percentKeywordsInTitle'] = round(numSame/feat['wordCount'], 2)
    
    feat['unigrams'] = sum(Counter(ngrams(text,1)).values())
    feat['bigrams'] = sum(Counter(ngrams(text,2)).values())
    feat['trigrams'] = sum(Counter(ngrams(text,3)).values())
    feat['fourgrams'] = sum(Counter(ngrams(text,4)).values())
    feat['fivegrams'] = sum(Counter(ngrams(text,5)).values())
    
    sentence = newsObject.postText[0]
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sentence)
    feat['posSent'] = ss['pos']
    feat['neuSent'] = ss['neu']
    feat['negSent'] = ss['neg']
    feat['compoundSent'] = ss['compound']
    
    paraSent = 0
    countPara = 0
    ss2 = {}
    for item in newsObject.targetParagraphs:
        feat['articleWords'] += len(item.split())
        text = nltk.word_tokenize(item)
        feat['unigramsArticle'] += sum(Counter(ngrams(text,1)).values())
        feat['bigramsArticle'] += sum(Counter(ngrams(text,2)).values())
        feat['trigramsArticle'] += sum(Counter(ngrams(text,3)).values())
        feat['fourgramsArticle'] += sum(Counter(ngrams(text,4)).values())
        feat['fivegramsArticle'] += sum(Counter(ngrams(text,5)).values())
        ss2 = sid.polarity_scores(item)
        countPara += 1
        paraSent += ss2['compound']
        
    if countPara > 0:
        paraSent = paraSent/countPara
    
    feat['sentDiffTitleBody'] = abs(feat['compoundSent'] - paraSent)
    
    
    #do noun, adverb, adjective, proper noun analysis for text bodies
        
    featTuple = (feat['wordCount'], feat['informal'], feat['beginsQuestion'], feat['beginsNum'], 
            feat['beginsThis'], feat['titleStopPerc'], feat['titleProperPerc'], 
            feat['posSent'], feat['neuSent'], feat['negSent'], feat['compoundSent'], feat['articleWords'], 
            feat['titlePercVerbs'], feat['unigrams'], feat['bigrams'], feat['trigrams'], feat['fourgrams'], 
            feat['fivegrams'], feat['percNouns'], feat['unigramsArticle'],
            feat['bigramsArticle'], feat['trigramsArticle'], feat['fourgramsArticle'], feat['fivegramsArticle'], 
            feat['percAdj'], feat['percAdv'], feat['percentKeywordsInTitle'], 
            feat['sentDiffTitleBody'], feat['hasContractions'], feat['hasAllCaps'],
            feat["averageWordLength"], feat['percSimilarToTitle'], feat['hasThis'], feat['hasQuestion'], feat['hasNumber'],
            feat['label'])
    newsObject.attributes = featTuple
    return newsObject
            

In [36]:
for item in instances:
    item = extractFeatures(item)



In [40]:
### dump to arff
features = [("word count", 'NUMERIC'),
            ("contains informal pronouns", 'NUMERIC'),
            ("Begins w/ question word", ['True', 'False']),
            ("Begins w/ number", ['True', 'False']),
            ("Begins with 'this'", ['True', 'False']),
            ("percent stop words", 'NUMERIC'),
            ("Percent proper nouns", 'NUMERIC'),
            ("Pos sent", 'NUMERIC'),
            ("Neu sent", 'NUMERIC'),
            ("Neg sent", 'NUMERIC'),
            ("Compound sent", 'NUMERIC'),
            ("Article Length", 'NUMERIC'),
            ("Percent verbs", 'NUMERIC'),
            ("Unigrams", 'NUMERIC'),
            ("Bigrams", 'NUMERIC'),
            ("Trigrams", 'NUMERIC'),
            ("Fourgrams", 'NUMERIC'),
            ("Fivegrams", 'NUMERIC'),
            ("Percent nouns", 'NUMERIC'),
            ("Unigrams article body", 'NUMERIC'),
            ("Bigrams article body", 'NUMERIC'),
            ("Trigrams article body", 'NUMERIC'),
            ("Fourgrams article body", 'NUMERIC'),
            ("Fivegrams article body", 'NUMERIC'),
            ("Percent adj", 'NUMERIC'),
            ("Percent adv", 'NUMERIC'),
            ("Percent keywords in title", 'NUMERIC'),
            ("Difference in sent body v. title", 'NUMERIC'),
            ('Has contractions', ['True', 'False']),
            ('Has all caps', ['True', 'False']),
            ("Average Word Length", 'NUMERIC'),
            ("Percent similar to title", "NUMERIC"),
            ("Has 'this", ['True', 'False']),
            ("Has question word", ['True', 'False']),
            ("Has number", ['True', 'False']),
            ("label", ['0', '1'])]
data = {}
data.setdefault('attributes', features)
data.setdefault('description', '')
data.setdefault('relation', 'clickbait_sample')
data.setdefault('data', [])
for item in instances:
    if item.attributes[35] == '1':
        print(item.postText[0])
        print(item.attributes)
    data['data'].append(item.attributes)

with open('sample_train.arff', 'w') as f:
    f.write(arff.dumps(data))

U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.
(12, 0, False, False, False, 0.17, 0.42, 0.201, 0.694, 0.104, 0.34, 695, 0.08, 15, 14, 13, 12, 11, 0.08, 805, 789, 774, 759, 744, 0.08, 0.0, 0.0, 0.3098375, False, False, 6.5, 0.75, False, True, False, '1')
Could light bulbs hurt your health? One company is now putting warning labels on its bulbs:
(16, 0, True, False, False, 0.31, 0.0, 0.0, 0.707, 0.293, -0.7003, 975, 0.12, 18, 17, 16, 15, 14, 0.19, 1123, 1102, 1081, 1060, 1039, 0.06, 0.06, 0.38, 0.6748809523809524, False, False, 4.75, 0.31, False, True, True, '1')
13 classic ’00s songs that were actually meant for other artists
(11, 0, False, True, False, 0.36, 0.0, 0.0, 1.0, 0.0, 0.0, 651, 0.18, 12, 11, 10, 9, 8, 0.09, 798, 777, 756, 735, 714, 0.18, 0.09, 0.0, 0.15832380952380953, False, False, 4.91, 0.18, False, False, True, '1')
Tourists detained in Malaysia for getting naked on sacred mountain:
(10, 0, False, False, False, 0.3, 0.1, 0.0, 0.76

Well, well, well...
(3, 0, False, False, False, 0.0, 0.0, 0.808, 0.192, 0.0, 0.4939, 351, 0.0, 6, 5, 4, 3, 2, 0.0, 398, 381, 364, 349, 337, 0.0, 0.33, 0.0, 0.41869999999999996, False, False, 5.67, 0.0, False, False, False, '1')
Quote of the day:
(4, 0, False, False, False, 0.5, 0.0, 0.0, 1.0, 0.0, 0.0, 1055, 0.0, 5, 4, 3, 2, 1, 0.5, 1216, 1197, 1178, 1159, 1140, 0.0, 0.0, 0.0, 0.4952210526315788, False, False, 3.5, 0.0, False, False, False, '1')
This used to be one of the best jobs in engineering. Not any more
(14, 0, False, False, True, 0.5, 0.0, 0.244, 0.756, 0.0, 0.6369, 676, 0.14, 15, 14, 13, 12, 11, 0.07, 777, 756, 735, 714, 695, 0.0, 0.07, 0.07, 0.4456714285714286, False, False, 3.71, 0.14, True, False, True, '1')
This pilot took to the sky to write a New Year's message
(12, 0, False, False, True, 0.33, 0.17, 0.0, 1.0, 0.0, 0.0, 211, 0.17, 13, 12, 11, 10, 9, 0.25, 255, 248, 241, 234, 227, 0.0, 0.0, 0.33, 0.25685714285714284, False, False, 3.75, 0.25, True, False, False, '1')
Gret

President Donald Trump: My first 100 days are "just about the most successful" in US history
(16, 0, False, False, False, 0.38, 0.25, 0.214, 0.786, 0.0, 0.624, 352, 0.06, 19, 18, 17, 16, 15, 0.06, 433, 423, 413, 403, 393, 0.12, 0.06, 0.06, 0.0639599999999999, False, False, 4.81, 0.75, False, False, True, '1')
19 people who probably should've just asked, "Do you want to go to prom?"
(14, 1, False, True, False, 0.43, 0.0, 0.091, 0.909, 0.0, 0.0772, 36, 0.43, 19, 18, 17, 16, 15, 0.0, 48, 44, 40, 36, 32, 0.0, 0.14, 0.0, 0.10579999999999999, True, False, 4.29, 0.14, False, True, True, '1')
Commentary: Everybody thinks they're losing
(5, 1, False, False, False, 0.2, 0.0, 0.0, 0.606, 0.394, -0.3818, 927, 0.2, 7, 6, 5, 4, 3, 0.4, 1076, 1058, 1040, 1022, 1004, 0.0, 0.0, 0.0, 0.28099444444444444, True, False, 7.8, 1.0, False, False, False, '1')
Here's how Scott Van Pelt paid tribute to laid-off ESPN employees on 'SportsCenter.'
(13, 0, False, False, False, 0.23, 0.31, 0.0, 1.0, 0.0, 0.0, 295, 0.

(10, 0, False, False, False, 0.3, 0.2, 0.193, 0.533, 0.273, -0.296, 699, 0.1, 10, 9, 8, 7, 6, 0.1, 817, 794, 771, 748, 725, 0.0, 0.0, 0.2, 0.21380869565217392, False, False, 6.8, 0.0, False, False, False, '1')
Vice President Mike Pence blasts the AP for publishing wife's email address
(12, 0, False, False, False, 0.17, 0.42, 0.0, 1.0, 0.0, 0.0, 421, 0.0, 13, 12, 11, 10, 9, 0.25, 498, 487, 476, 465, 454, 0.0, 0.0, 0.25, 0.3146454545454546, False, False, 5.33, 1.0, False, False, False, '1')
Trump's trade threat is uniting the rest of the world, top EU official says
(14, 0, False, False, False, 0.29, 0.14, 0.105, 0.698, 0.198, -0.3818, 534, 0.0, 16, 15, 14, 13, 12, 0.36, 644, 631, 618, 605, 592, 0.07, 0.0, 0.0, 0.4629076923076923, False, False, 4.43, 0.14, False, False, False, '1')
Who ya got?

#UFC209 predictions ahead of tonight’s fights
(8, 1, False, False, False, 0.25, 0.25, 0.0, 0.748, 0.252, -0.4019, 1121, 0.12, 13, 12, 11, 10, 9, 0.12, 1329, 1271, 1213, 1171, 1129, 0.12, 0.12, 0.0,

There's a weird noise in Sweden and no one is sure where it's coming from
(15, 1, False, False, False, 0.53, 0.07, 0.134, 0.64, 0.227, -0.1531, 605, 0.0, 17, 16, 15, 14, 13, 0.13, 683, 669, 655, 641, 627, 0.13, 0.0, 0.2, 0.14117857142857143, True, False, 3.93, 0.13, False, True, False, '1')
Ex-NBA player arrested in alleged kidnapping, robbery, assault  via @NBCSports
(11, 0, False, False, False, 0.09, 0.09, 0.0, 0.537, 0.463, -0.7845, 3454, 0.09, 13, 12, 11, 10, 9, 0.45, 4069, 4001, 3933, 3865, 3797, 0.09, 0.0, 0.09, 0.9466014084507042, False, False, 6.18, 0.45, False, False, False, '1')
Despite decline in Americans identifying as Christians, Congress is still overwhelmingly Christian, report says
(14, 0, False, False, False, 0.21, 0.07, 0.0, 0.897, 0.103, -0.128, 543, 0.0, 16, 15, 14, 13, 12, 0.14, 638, 624, 610, 596, 582, 0.07, 0.14, 0.07, 0.2551285714285714, False, False, 7.0, 0.07, False, False, False, '1')
Anthony Bourdain bashes fellow "privileged Eastern liberals" for making Tr

In [38]:
text1 = nltk.word_tokenize("you yourself your you're")
tokenizedList = nltk.pos_tag(text1)
print(tokenizedList)
text = nltk.word_tokenize(instances[100].postText[0])
bigrams = ngrams(text,1)
print(Counter(bigrams).values())


[('you', 'PRP'), ('yourself', 'PRP'), ('your', 'PRP$'), ('you', 'PRP'), ("'re", 'VBP')]
dict_values([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


["'tis",
 "'twas",
 "ain't",
 "aren't",
 "can't",
 "could've",
 "couldn't",
 "didn't",
 "doesn't",
 "don't",
 "hasn't",
 "he'd",
 "he'll",
 "he's",
 "how'd",
 "how'll",
 "how's",
 "i'd",
 "i'll",
 "i'm",
 "i've",
 "isn't",
 "it's",
 "might've",
 "mightn't",
 "must've",
 "mustn't",
 "shan't",
 "she'd",
 "she'll",
 "she's",
 "should've",
 "shouldn't",
 "that'll",
 "that's",
 "there's",
 "they'd",
 "they'll",
 "they're",
 "they've",
 "wasn't",
 "we'd",
 "we'll",
 "we're",
 "weren't",
 "what'd",
 "what's",
 'when',
 "when'd",
 "when'll",
 "when's",
 "where'd",
 "where'll",
 "where's",
 "who'd",
 "who'll",
 "who's",
 "why'd",
 "why'll",
 "why's",
 "won't",
 "would've",
 "wouldn't",
 "you'd",
 "you'll",
 "you're",
 "you've"]